# ML Project Fine Food Reviews
Made by: Saurav Lama<br>
Section: KM058<br>
Roll Number: A13<br>
Submitted to: Pooja Rana<br>

 Id: For reach review <br> <Product id- Id of the product<br> UserId:- Name of the user<br> ProfileName<br> 
            Helpfulness NUmerator Means this is the people who think this review is helpful<br> Score means 1 to 5 scale <br> Time ; means on which time the review was given<br> Text means what written on the review<br> Summary: breif intro about their review<br>

Now i dont want the score i want to predict whether the review is positive or negative. So what i can do is simple use a new column and put positive if score is more than 3 and negative if less than 3

In [5]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sqlite3
import nltk
import seaborn as sns



In [3]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

In [6]:
con=sqlite3.connect('database.sqlite')

In [7]:
dataBase=pd.read_sql_query("""
Select *
from Reviews
where Score !=3
""", con)

In [8]:
type(dataBase)

pandas.core.frame.DataFrame

In [5]:
def change(n):
    if n>3:
        return 'Good Review'
    else:
        return 'Bad Review'

Score=dataBase['Score']
Reviews=Score.map(change)


In [6]:
dataBase['Score']=Reviews

In [7]:
dataBase.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,Good Review,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,Bad Review,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,Good Review,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,Bad Review,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,Good Review,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [8]:
SortedDatabase= dataBase.sort_values(by=['ProductId'], ascending=True,)

In [9]:
SortedDatabase.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,Good Review,939340800,EVERY book is educational,this witty little book makes my son laugh at l...
138688,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,Good Review,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc..."
138689,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,Good Review,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...
138690,150508,0006641040,AZGXZ2UUK6X,"Catherine Hallberg ""(Kate)""",1,1,Good Review,1076025600,a good swingy rhythm for reading aloud,This is a great little book to read aloud- it ...
138691,150509,0006641040,A3CMRKGE0P909G,Teresa,3,4,Good Review,1018396800,A great way to learn the months,This is a book of poetry about the months of t...


In [10]:
finalDatabase=SortedDatabase.drop_duplicates(subset={"UserId","ProfileName","Time","Text"},keep="first",inplace=False)

In [11]:

a=SortedDatabase['Id'].size
print(a)

525814


In [12]:
b=finalDatabase['Id'].size
print(b)

364173


In [59]:
print(c)

161641


In [69]:
(c*1.0)/(a*1.0)*100

30.74109856337032

In [79]:
finalDatabase[finalDatabase.HelpfulnessNumerator > finalDatabase.HelpfulnessDenominator]

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
59301,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,Good Review,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...
41159,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,Good Review,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...


In [13]:
finalDatabase=finalDatabase[finalDatabase.HelpfulnessNumerator <= finalDatabase.HelpfulnessDenominator]

In [24]:
finalDatabase=finalDatabase.head(10000)

BAG OF WORDS now we will convert the text feild into a vector

In [25]:
#we are basically making a variable of the type CountVectorizer
countVect= CountVectorizer()
finalCount=countVect.fit_transform(finalDatabase['Text'].values)

In [26]:
type(finalCount)

scipy.sparse.csr.csr_matrix

this is a type of sparse matrix

Now finalCount is a matrix with 364171 rows and 115281 columns where every columns represents a peice of word or each dimension

In this review we have a lots of html tag and commas and . or # and convert them into lower case and do lemmentization, stemming and using portball stemming <br> and even remove stopwords

In [108]:
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

stop=set(stopwords.words("english"))#case-sensitive
sno=nltk.stem.SnowballStemmer('english') #initializing snowball stemmer
print(stop)
def cleantags(sen):
    clean=re.compile('<.*?>')
    cleaned=re.sub(clean,'',sen)
    return cleaned
def cleanpunctuations(sen):
    cleann=re.sub(r'[?|!|\'|"|#]',r'',sen)
    cleann=re.sub(r'[,|.|)|(|\|/]',r'',cleann)
    return cleann


{'are', "doesn't", 'don', 'while', "should've", 'above', 'up', 'didn', 'by', 'when', 'herself', 'each', 'haven', 'him', 'this', 'only', "didn't", 'down', 'same', "mightn't", 'who', 'some', 'aren', "won't", 'into', 'am', 'few', 'between', 'y', 'being', 'do', 'because', "it's", "wasn't", 'which', 'll', 'mustn', 'ourselves', "she's", "wouldn't", 'such', 'just', 'mightn', 'so', "that'll", 'a', 'with', 'in', "shouldn't", 'then', 'should', 'further', 'under', 'hadn', 'o', 'yours', 'ma', "you've", "you'd", "couldn't", 'them', 'below', 'is', 'at', 'yourself', 'about', 'why', 'she', 'ours', 'through', 'did', 'ain', 'that', 'off', 'have', 'here', 'needn', 'me', 'most', 'shan', 'not', 'doing', 'we', 'their', 'how', "you'll", 'his', 'wasn', 'doesn', 'than', "shan't", 'from', 'wouldn', 'won', 'been', 'theirs', 'had', "hasn't", 'be', 'any', 'those', 'whom', 'both', 'couldn', 'can', 'after', 'weren', 'myself', 'again', "you're", 'too', 'he', 'of', 'before', "aren't", "needn't", 'd', 'no', "isn't", 'h

In [99]:
print(Snow.stem('Whatever'))

whatev


In [111]:
i=0
strin=''
positive=[]
negative=[]
finalstring=[]
s=''
for sent in finalDatabase['Text'].values:
    filtered_sentences=[]
    sent=cleantags(sent)
    for a in sent.split():
        for cleaned_words in cleanpunctuations(a).split():
            if((cleaned_words.isalpha())& (len(cleaned_words)>2)):
                if(cleaned_words.lower() not in stop):
                    s=(sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sentences.append(s)
                    if(finalDatabase['Score'].values)[i]=='Good Review':
                        positive.append(s) #list of all words used
                    if(finalDatabase['Score'].values)[i]== 'Bad Review':
                        negative.append(s)
                else:
                    continue
            else:
                continue
    strin = b" ".join(filtered_sentences) 
    finalstring.append(strin)
    i+=1

In [116]:
finalDatabase['CleanedText']=finalstring

conn=sqlite3.connect('final.sqlite')
c=conn.cursor()
conn.text_factory=str
finalDatabase.to_sql('Reviews',conn,schema=None)

In [117]:
finalDatabase['CleanedText'].head()

138706    b'witti littl book make son laugh loud recit c...
138688    b'grew read sendak book watch realli rosi movi...
138689    b'fun way children learn month year learn poem...
138690    b'great littl book read nice rhythm well good ...
138691    b'book poetri month year goe month cute littl ...
Name: CleanedText, dtype: object

In [118]:
freqofpositive=nltk.FreqDist(positive)

In [119]:
freqofpositive.most_common(20)

[(b'like', 138328),
 (b'tast', 126012),
 (b'good', 109815),
 (b'love', 106494),
 (b'flavor', 106379),
 (b'use', 102864),
 (b'great', 101115),
 (b'one', 94386),
 (b'product', 88456),
 (b'tri', 85095),
 (b'tea', 81808),
 (b'coffe', 76651),
 (b'make', 74450),
 (b'get', 71691),
 (b'food', 63018),
 (b'would', 55117),
 (b'buy', 53557),
 (b'time', 53550),
 (b'realli', 52388),
 (b'eat', 51316)]

In [121]:
count_vect=CountVectorizer(ngram_range=(1,2))
bigramcount=count_vect.fit_transform(finalDatabase['CleanedText'].values)

In [123]:
bigramcount.get_shape()

(364171, 3417839)

In [124]:
type(bigramcount)

scipy.sparse.csr.csr_matrix

TFIDF

In [133]:
tfidf=TfidfVectorizer(ngram_range=(1,2))
finaltfidf=tfidf.fit_transform(finalDatabase['Text'].values)

In [134]:
finaltfidf.get_shape()

(364171, 2910192)

In [136]:
features= tfidf.get_feature_names()

In [138]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle
import gensim


In [162]:
i=0
list_of_sent=[]
for sent in finalDatabase['Text'].values:
    filtered_sentence=[]
    sent=cleantags(sent)
    for w in sent.split():
        for cleaned_words in cleanpunctuations(w).split():
            if(cleaned_words.isalpha()):
                filtered_sentence.append(cleaned_words.lower())
            else:
                continue
    list_of_sent.append(filtered_sentence)

In [163]:
print(list_of_sent[0])

['this', 'witty', 'little', 'book', 'makes', 'my', 'son', 'laugh', 'at', 'loud', 'i', 'recite', 'it', 'in', 'the', 'car', 'as', 'were', 'driving', 'along', 'and', 'he', 'always', 'can', 'sing', 'the', 'refrain', 'hes', 'learned', 'about', 'whales', 'india', 'drooping', 'i', 'love', 'all', 'the', 'new', 'words', 'this', 'book', 'introduces', 'and', 'the', 'silliness', 'of', 'it', 'all', 'this', 'is', 'a', 'classic', 'book', 'i', 'am', 'willing', 'to', 'bet', 'my', 'son', 'will', 'still', 'be', 'able', 'to', 'recite', 'from', 'memory', 'when', 'he', 'is', 'in', 'college']


In [164]:
w2vm=gensim.models.Word2Vec(list_of_sent,min_count=5,size=50,workers=4)

Now we are going to store all the words in a dictionary format

In [165]:
w2vm.wv.most_similar('love')

[('adore', 0.8551236391067505),
 ('enjoy', 0.7277843952178955),
 ('luv', 0.7209630012512207),
 ('loves', 0.6727988719940186),
 ('loved', 0.6519018411636353),
 ('appreciate', 0.6151466369628906),
 ('enjoys', 0.6128970384597778),
 ('enjoyed', 0.5953759551048279),
 ('loving', 0.5769585967063904),
 ('despise', 0.5767354369163513)]

In [166]:
import pandas as pd
import seaborn as sn
import numpy as np
import matplotlib.pyplot as plt

In [27]:
ok=pd.DataFrame.sparse.from_spmatrix(finalCount)

In [ ]:
from sklearn.manifold import TSNE
model=TSNE(n_components=2,random_state=0, perplexity=20,n_iter=1000) #random state should be 0 to want sample output if runned for
#default perp=30, iteration=1000
tsne_data=model.fit_transform(ok)